In [1]:
import pandas as pd

# Carregar o arquivo CSV
file_path = "processed_data_categorized.csv"
df = pd.read_csv(file_path)

# Remover as colunas especificadas
df = df.drop(columns=['validated_sentiment', 'entities', 'doctor_name', 'hospital_name', 'location'])

# Salvar o DataFrame de volta no arquivo CSV
df.to_csv(file_path, index=False)

print("Colunas removidas e arquivo salvo com sucesso.")


FileNotFoundError: [Errno 2] No such file or directory: 'processed_data_categorized.csv'

In [ ]:
import pandas as pd

# Carregar o arquivo CSV
file_path = "processed_data_categorized.csv"
df = pd.read_csv(file_path)

# Determinar o ponto de divisão
half_size = len(df) // 2

# Dividir a base em duas partes iguais
df_part1 = df.iloc[:half_size]
df_part2 = df.iloc[half_size:]

# Salvar as duas partes em arquivos separados
df_part1_path = "processed_data_part1.csv"
df_part2_path = "processed_data_part2.csv"

df_part1.to_csv(df_part1_path, index=False)
df_part2.to_csv(df_part2_path, index=False)

print(f"Base de dados dividida em duas partes:\n{df_part1_path}\n{df_part2_path}")


Base de dados dividida em duas partes:
processed_data_part1.csv
processed_data_part2.csv


In [2]:
import pandas as pd

# Carregar as duas partes
df_part1 = pd.read_csv("processed_data_part1.csv")
df_part2 = pd.read_csv("processed_data_part2.csv")

# 1. Verificar se as colunas são iguais
columns_equal = df_part1.columns.equals(df_part2.columns)
print(f"As colunas são iguais: {columns_equal}")

# 2. Comparar a quantidade de linhas
rows_part1 = len(df_part1)
rows_part2 = len(df_part2)
print(f"Quantidade de linhas em Part 1: {rows_part1}")
print(f"Quantidade de linhas em Part 2: {rows_part2}")

# 3. Verificar valores nulos em ambas as partes
nulls_part1 = df_part1.isnull().sum().sum()
nulls_part2 = df_part2.isnull().sum().sum()
print(f"Valores nulos em Part 1: {nulls_part1}")
print(f"Valores nulos em Part 2: {nulls_part2}")

# 4. Verificar duplicados em ambas as partes
duplicates_part1 = df_part1.duplicated().sum()
duplicates_part2 = df_part2.duplicated().sum()
print(f"Linhas duplicadas em Part 1: {duplicates_part1}")
print(f"Linhas duplicadas em Part 2: {duplicates_part2}")

# 5. Verificar se as colunas numéricas têm a mesma média e desvio padrão
numerical_columns = df_part1.select_dtypes(include=['number']).columns
stats_part1 = df_part1[numerical_columns].describe().loc[['mean', 'std']]
stats_part2 = df_part2[numerical_columns].describe().loc[['mean', 'std']]

# Comparar estatísticas entre as partes
stats_comparison = stats_part1.compare(stats_part2)
print("Comparação de estatísticas (média e desvio padrão) entre Part 1 e Part 2:")
print(stats_comparison)

# 6. Verificar se as colunas categóricas têm a mesma distribuição
categorical_columns = df_part1.select_dtypes(include=['object']).columns
for column in categorical_columns:
    dist_part1 = df_part1[column].value_counts(normalize=True, sort=False)
    dist_part2 = df_part2[column].value_counts(normalize=True, sort=False)
    
    # Realizar a comparação ignorando os índices
    distribution_comparison = pd.concat([dist_part1, dist_part2], axis=1, keys=['Part1', 'Part2']).fillna(0)
    distribution_comparison['Difference'] = distribution_comparison['Part1'] - distribution_comparison['Part2']
    
    # Mostrar somente diferenças significativas
    differences = distribution_comparison[distribution_comparison['Difference'].abs() > 0.01]
    if not differences.empty:
        print(f"Diferença na distribuição categórica para a coluna '{column}':")
        print(differences)

print("Auditoria concluída.")


As colunas são iguais: True
Quantidade de linhas em Part 1: 22425
Quantidade de linhas em Part 2: 22426
Valores nulos em Part 1: 3
Valores nulos em Part 2: 15
Linhas duplicadas em Part 1: 5
Linhas duplicadas em Part 2: 3
Comparação de estatísticas (média e desvio padrão) entre Part 1 e Part 2:
     original_sentiment           cleaned_sentiment            \
                   self     other              self     other   
mean           0.096869  0.190905          0.082053  0.210397   
std            0.615894  0.638947          0.546750  0.605032   

     sentiment_difference           original_length               \
                     self     other            self        other   
mean             0.211678  0.190481      972.619621  1070.123874   
std              0.309741  0.317607     1439.412095  1784.971623   

     cleaned_length               
               self        other  
mean     512.567982   599.073486  
std      871.299908  1197.958638  
Diferença na distribuição categ

In [5]:
df_final = pd.concat([df_part1, df_part2], ignore_index=True)

df_final.head()


,country_code,type,created_at,content_en,cleaned_content,original_sentiment,cleaned_sentiment,sentiment_difference,original_length,cleaned_length,category_keywords,category_bert,issue_type,urgency_level,resolution_suggestion
0,br,opinion,2024-04-06,"A doctor who is not very inclusive, it seems t...",doctor not_very inclusive seems want immediate...,-0.9209,-0.9126,0.0083,1018,586,Scheduling Issue,Wait Time Issue,Critical,High,Immediate Action Required
1,br,opinion,2024-04-12,Although the online schedule showed availabili...,although online schedule showed availability t...,0.0000,0.0000,0.0000,154,110,Scheduling Issue,Facility Issue,Non-Critical,Low,Monitor
2,br,opinion,2024-04-22,Didn't answer my wife. After traveling more th...,answer wife traveling three hour even though c...,-0.8079,-0.5719,0.2360,246,156,General Issue,Wait Time Issue,Critical,High,Immediate Action Required
3,br,opinion,2024-04-27,\nHe works at the Santo Amaro rehabilitation c...,work santo amaro rehabilitation center not_pre...,-1.0000,-0.8027,0.1973,624,416,Wait Time Issue,Facility Issue,Critical,High,Immediate Action Required
4,br,opinion,2024-04-29,This psychologist canceled the 1st appointment...,psychologist canceled st appointment wa late s...,0.3480,-0.8729,1.2209,636,385,Scheduling Issue,Wait Time Issue,Critical,High,Immediate Action Required


In [6]:
# Utilizando o DataFrame df_final criado anteriormente

# 1. Verificar o intervalo de datas
date_min = pd.to_datetime(df_final['created_at']).min()
date_max = pd.to_datetime(df_final['created_at']).max()

date_range_valid = date_min == '2024-04-01' and date_max == '2024-06-30'
print(f"Intervalo de datas está correto: {date_range_valid}")
print(f"Intervalo de datas: {date_min} a {date_max}")

# 2. Verificar se está organizado em 12 semanas
df_final['week_number'] = pd.to_datetime(df_final['created_at']).dt.isocalendar().week
unique_weeks = df_final['week_number'].nunique()
weeks_valid = unique_weeks == 12
print(f"Dataset organizado em 12 semanas: {weeks_valid}")
print(f"Número de semanas únicas: {unique_weeks}")

# 3. Verificar a presença dos três países
expected_countries = {'es', 'de', 'br'}
actual_countries = set(df_final['country_code'].unique())
countries_valid = actual_countries == expected_countries
print(f"Países presentes estão corretos: {countries_valid}")
print(f"Países encontrados: {actual_countries}")

# 4. Verificar o total de registros
total_records = len(df_final)
total_records_valid = total_records == 43251
print(f"Total de registros é 43.251: {total_records_valid}")
print(f"Total de registros: {total_records}")


Intervalo de datas está correto: False
Intervalo de datas: 2024-04-01 00:00:00 a 2024-06-30 00:00:00
Dataset organizado em 12 semanas: False
Número de semanas únicas: 13
Países presentes estão corretos: True
Países encontrados: {'es', 'br', 'de'}
Total de registros é 43.251: False
Total de registros: 44851
